This colab will generate the timeline files.  

Details: https://model.earth/data-commons/docs/data

TO DO: Starting with python Abhishe created, create a function called timelineGeneration that generates the timeline csv files. Pass it a string containing DCID files pulled from our Google Sheet UN Goal tabs.

TO DO: Pull timeline DCIDs from Google Sheet

TO DO: Save to a Github fork using steps documented at bottom of page above. Timelines can reside in a timelines/[goal] subfolder. Example:
https://github.com/ModelEarth/data-commons/tree/main/docs/data/timelines/air

Sample output:
https://github.com/ModelEarth/community-data/blob/master/locations/datacommons/country.csv

If needs to send file to github: https://model.earth/localsite/info/data/map-filters/



Question: Unable to find worldBank/EN_ATM_CO2E_KT	Annual Emission (KT), may I ask what specific item does it refers to?

# Import

In [7]:
pip install datacommons_pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pycountry

In [9]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import datacommons_pandas as dc
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
import pycountry

In [24]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
client = gspread.authorize(creds)

ModuleNotFoundError: No module named 'google.colab'

In [22]:
from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [21]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
client = gspread.authorize(creds)


ModuleNotFoundError: No module named 'google.colab'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [5]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2
import gspread
from google.colab import auth
from google.auth import default


ModuleNotFoundError: No module named 'google.colab'

# Read Google sheet

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open("UN Goals")


In [ ]:
worksheet = spreadsheet.get_worksheet(1)
data = worksheet.get_all_values()
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8
0,Topic,Subtopic,DCID,Title,IsUsable,Scope,StartYear,EndYear,Source
1,Emissions,CO2,Annual_Emissions_CarbonDioxide_NonBiogenic,Non Biogenic Emission,,"State, County, Zip",2010,2021,https://datacommons.org/tools/statvar#sv=
2,Emissions,CO2,Annual_Emissions_CarbonDioxide_Biogenic,Biogenic Emission,,"State, County, Zip",2010,2021,
3,Emissions,CO2,worldBank/EN_ATM_CO2E_KT,Annual Emission (KT),,Country,1990,2020,
4,Emissions,CO2,Annual_Amount_Emissions_CarbonDioxide,Annual Emission,(A lot of missing values),Country,1960,2021,
5,Emissions,Methane,Annual_Emissions_Methane_NonBiogenic,Non Biogenic Emission,,"State, County, Zip",2010,2021,
6,Emissions,Methane,,,,,,,
7,Emissions,Greenhouse Gas,Annual_Emissions_GreenhouseGas_NonBiogenic,Non Biogenic Emission,,"State, County, Zip",,,
8,Emissions,Lead,Annual_Emissions_GreenhouseGas_LeadProduction_...,Non Biogenic Lead Production,,"State, County, Zip",,,


In [ ]:
selected_columns=df.iloc[:, [2, 5]]

In [ ]:
selected_columns.columns = ['DCID', 'Scope']
country_dcid = selected_columns[selected_columns['Scope'] == 'Country']['DCID']
state_dcid = selected_columns[selected_columns['Scope'].str.contains('State', na=False)]['DCID']
county_dcid = selected_columns[selected_columns['Scope'].str.contains('County', na=False)]['DCID']

# Pull Country Timeline Data

In [ ]:
def get_country_ids():
    country_dcids = dc.get_places_in(['Earth'], 'Country')
    country_names = {dcid: dc.get_property_values([dcid], 'name')[dcid][0] for dcid in country_dcids['Earth']}
    return country_names

country_ids_data = get_country_ids()
country_ids_data

{'country/ABW': 'Aruba',
 'country/AFG': 'Afghanistan',
 'country/AGO': 'Angola',
 'country/AIA': 'Anguilla',
 'country/ALA': 'Åland Islands',
 'country/ALB': 'Albania',
 'country/AND': 'Andorra',
 'country/ANT': 'Netherlands Antilles',
 'country/ARE': 'United Arab Emirates',
 'country/ARG': 'Argentina',
 'country/ARM': 'Armenia',
 'country/ASM': 'American Samoa',
 'country/ATA': 'Antarctica',
 'country/ATB': 'British Antarctic Territory',
 'country/ATF': 'French Southern Territories',
 'country/ATG': 'Antigua and Barbuda',
 'country/ATN': 'Dronning Maud Land',
 'country/AUS': 'Australia',
 'country/AUT': 'Austria',
 'country/AZE': 'Azerbaijan',
 'country/BDI': 'Burundi',
 'country/BEL': 'Belgium',
 'country/BEN': 'Benin',
 'country/BES': 'Bonaire, Sint Eustatius and Saba',
 'country/BFA': 'Burkina Faso',
 'country/BGD': 'Bangladesh',
 'country/BGR': 'Bulgaria',
 'country/BHR': 'Bahrain',
 'country/BHS': 'Bahamas',
 'country/BIH': 'Bosnia and Herzegovina',
 'country/BLM': 'Saint Barthé

In [ ]:
def get_country_code(country_name, existing_codes):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.alpha_2
    except LookupError:
      #note that there are some counties only exist in datacommon but not in pycountry
        print(country_name)
        code = country_name[:2].upper()
        if code in existing_codes:
            code = (country_name[0] + country_name[-1]).upper()
        return code

def create_country_dataframe(country_ids_data):
    countries = []
    existing_codes = {country.alpha_2 for country in pycountry.countries}

    for dcid, name in country_ids_data.items():
        country_code = get_country_code(name, existing_codes)
        countries.append((country_code, name))
        existing_codes.add(country_code)

    country_df = pd.DataFrame(countries, columns=['CountryCode', 'CountryName'])
    return country_df

country_data = create_country_dataframe(country_ids_data)
country_data

Netherlands Antilles
British Antarctic Territory
Dronning Maud Land
Brunei
Congo [DRC]
Congo [Republic]
Cape Verde
Canton and Enderbury Islands
Falkland Islands
Metropolitan France
Macau
Saint Martin
Macedonia [FYROM]
Pitcairn Islands
Palestinian Territories
Russia
Saint Helena
São Tomé and Príncipe
Sint Maarten
East Timor
Turkey
U.S. Minor Outlying Islands
Vatican City
U.S. Virgin Islands
Kosovo
Yugoslavia


,CountryCode,CountryName
0,AW,Aruba
1,AF,Afghanistan
2,AO,Angola
3,AI,Anguilla
4,AX,Åland Islands
...,...,...
251,YE,Yemen
252,YU,Yugoslavia
253,ZA,South Africa
254,ZM,Zambia


In [ ]:
def timelineGenerationCountry(stat_var):
    country_data = create_country_dataframe(country_ids_data)
    country_ids = get_country_ids()
    data_dict = {}
    for dcid, name in country_ids.items():
        try:
            data = dc.get_stat_series(dcid, stat_var)
            if data:
                data_dict[name] = data
        except Exception as e:
            print(f"Failed to fetch data for {name} ({dcid}): {e}")
    result = pd.DataFrame.from_dict(data_dict, orient='index').fillna(0)
    result.reset_index(inplace=True)
    result.rename(columns={'index': 'CountryName'}, inplace=True)
    result = pd.merge(country_data, result, on='CountryName', how='right')
    year_cols = sorted(col for col in result.columns if col.isdigit())
    cols = ['CountryName', 'CountryCode'] + year_cols
    result = result[cols]

    filename = f"{stat_var}_country_data_timeline.csv"
    result.to_csv(filename, index=False)
    print(f"CSV file '{filename}' has been created successfully.")

    return result


In [ ]:
timelineGenerationCountry('Annual_Amount_Emissions_CarbonDioxide')

Netherlands Antilles
British Antarctic Territory
Dronning Maud Land
Brunei
Congo [DRC]
Congo [Republic]
Cape Verde
Canton and Enderbury Islands
Falkland Islands
Metropolitan France
Macau
Saint Martin
Macedonia [FYROM]
Pitcairn Islands
Palestinian Territories
Russia
Saint Helena
São Tomé and Príncipe
Sint Maarten
East Timor
Turkey
U.S. Minor Outlying Islands
Vatican City
U.S. Virgin Islands
Kosovo
Yugoslavia
CSV file 'Annual_Amount_Emissions_CarbonDioxide_country_data_timeline.csv' has been created successfully.


,CountryName,CountryCode,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Angola,AO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,16437322,19280474,20882421,21519901,19331502,16714959,17200100,18916730,15141577,0.00
1,Albania,AL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3501785,3666407,4143032,3842930,3692745,4392695,4388485,4065651,3392703,4055148.30
2,United Arab Emirates,AE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,168664956,177131791,177700469,186367920,192863757,183983704,166074879,177884345,180796048,0.00
3,Argentina,AR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,172634382,177376004,173869592,179754063,178071086,173941156,171202444,162244127,149024062,165767394.00
4,Armenia,AM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5438573,5242694,5235360,5102732,4912583,5165799,5397060,5854471,6333916,6858307.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,South Africa,ZA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,422291527,432315445,443331367,419595099,419949543,429701733,434121305,441115684,388114479,0.00
139,Zambia,ZM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3418494,3534140,3949822,4259091,4540305,5954336,6791235,6654690,6677107,0.00
140,Zimbabwe,ZW,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,11388969,11753572,11514750,11403596,9989077,9312083,10511915,9712632,7816521,0.00
141,Laos,LA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2706166,3336481,3504599,7692946,14402240,17551512,17831771,17183929,16827882,0.00


# Pull States Timeline Data

In [ ]:
stateDict = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California", "CO": "Colorado",
    "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana",
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota",
    "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming",
    "DC": "District of Columbia",
    # US Territories
    "AS": "American Samoa", "GU": "Guam", "MP": "Northern Mariana Islands", "PR": "Puerto Rico", "VI": "Virgin Islands of the U.S."
}

stateData = pd.DataFrame(list(stateDict.items()),columns = ['State','StateName'])
stateData.head()

,State,StateName
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [ ]:
def get_state_ids():
  state_dcids = dc.get_places_in(['country/USA'], 'State')
  state_names = [dc.get_property_values([name], 'name') for name in state_dcids['country/USA']]

  data = {}
  for entry in state_names:
      for k,v in entry.items():
          data[k] = v[0]

  # Adding Rest 4 US Territories
  for k,v in dc.get_property_values(['geoId/60','geoId/66','geoId/69','geoId/78'], 'name').items():
      data[k] = v[0]
  return data
  state_ids_data = get_state_ids()

In [ ]:
def timelineGenerationState(stat_var):
    stateData = pd.DataFrame(list(stateDict.items()),columns = ['State','StateName'])
    state_ids = get_state_ids()
    state_id_data = pd.DataFrame(list(state_ids.items()), columns=['StateID', 'StateName'])
    existing_codes = set(state_id_data['StateID'])
    data_dict = {}
    for dcid, name in state_ids.items():
        try:
            data = dc.get_stat_series(dcid, stat_var)
            if data:
                data_dict[name] = data
        except Exception as e:
            print(f"Failed to fetch data for {name} ({dcid}): {e}")
    result = pd.DataFrame.from_dict(data_dict, orient='index').fillna(0)
    result.reset_index(inplace=True)
    result.rename(columns={'index': 'StateName'}, inplace=True)
    result = pd.merge(stateData, result, on='StateName', how='right')
    year_cols = sorted(col for col in result.columns if col.isdigit())
    cols = ['StateName', 'State'] + year_cols
    result = result[cols]
    filename = f"{stat_var}_state_data_timeline.csv"
    result.to_csv(filename, index=False)
    print(f"CSV file '{filename}' has been created successfully.")
    return result



In [ ]:
for i in state_dcid:
  stat_var=i
  timelineGenerationState(i)

CSV file 'Annual_Emissions_CarbonDioxide_NonBiogenic_state_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_CarbonDioxide_Biogenic_state_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_Methane_NonBiogenic_state_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_GreenhouseGas_NonBiogenic_state_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_GreenhouseGas_LeadProduction_NonBiogenic_state_data_timeline.csv' has been created successfully.


# Pull County Timeline Data

In [ ]:
def get_county_ids():
    state_dcids = dc.get_places_in(['country/USA'], 'State')['country/USA']
    county_dcids = dc.get_places_in(state_dcids, 'County')
    county_names = {}
    for state_dcid, counties in county_dcids.items():
        for county_dcid in counties:
            name = dc.get_property_values([county_dcid], 'name').get(county_dcid, [''])[0]
            county_names[county_dcid] = name
    return county_names

county_ids_data = get_county_ids()

def create_county_dataframe(county_ids_data):
    counties = []
    for dcid, name in county_ids_data.items():
        counties.append((dcid, name))
    county_df = pd.DataFrame(counties, columns=['CountyID', 'CountyName'])
    return county_df

county_data = create_county_dataframe(county_ids_data)

def timelineGenerationCounty(stat_var):
    county_ids = get_county_ids()
    county_data = create_county_dataframe(county_ids)
    data_dict = {}

    for dcid, name in county_ids.items():
        try:
            data = dc.get_stat_series(dcid, stat_var)
            if data:
                data_dict[name] = data
        except Exception as e:
            print(f"Failed to fetch data for {name} ({dcid}): {e}")
    result = pd.DataFrame.from_dict(data_dict, orient='index').fillna(0)
    result.reset_index(inplace=True)
    result.rename(columns={'index': 'CountyName'}, inplace=True)
    result = pd.merge(county_data, result, on='CountyName', how='right')

    year_cols = sorted(col for col in result.columns if col.isdigit())
    cols = ['CountyName', 'CountyID'] + year_cols
    result = result[cols]
    filename = f"{stat_var}_county_data_timeline.csv"
    result.to_csv(filename, index=False)
    print(f"CSV file '{filename}' has been created successfully.")

    return result



In [ ]:
for i in state_dcid:
  stat_var=i
  timelineGenerationCounty(i)

CSV file 'Annual_Emissions_CarbonDioxide_NonBiogenic_county_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_CarbonDioxide_Biogenic_county_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_Methane_NonBiogenic_county_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_GreenhouseGas_NonBiogenic_county_data_timeline.csv' has been created successfully.
CSV file 'Annual_Emissions_GreenhouseGas_LeadProduction_NonBiogenic_county_data_timeline.csv' has been created successfully.


Zipcode


In [ ]:
def get_zipcode_ids():
    zipcode_dcids = df.iloc[:, 2].tolist()
    zipcodes = df.iloc[:, 3].tolist()
    return dict(zip(zipcode_dcids, zipcodes))

def create_zipcode_dataframe(zipcode_ids_data):
    zipcodes = []
    for dcid, zipcode in zipcode_ids_data.items():
        zipcodes.append((dcid, zipcode))
    zipcode_df = pd.DataFrame(zipcodes, columns=['DCID', 'ZIPCode'])
    return zipcode_df

def timelineGenerationZip(stat_var):
    zipcode_ids = get_zipcode_ids()
    zipcode_data = create_zipcode_dataframe(zipcode_ids)
    data_dict = {}

    for dcid, zipcode in zipcode_ids.items():
        try:
            data = dc.get_stat_series(dcid, stat_var)
            if data:
                data_dict[zipcode] = data
        except Exception as e:
            print(f"Failed to fetch data for ZIP code {zipcode} (DCID {dcid}): {e}")

    result = pd.DataFrame.from_dict(data_dict, orient='index').fillna(0)
    result.reset_index(inplace=True)
    result.rename(columns={'index': 'ZIPCode'}, inplace=True)
    result = pd.merge(zipcode_data, result, on='ZIPCode', how='right')

    year_cols = sorted(col for col in result.columns if col.isdigit())
    cols = ['ZIPCode', 'DCID'] + year_cols
    result = result[cols]

    return result

# Example usage
stat_var = 'Count_Person'
zipcode_result = timelineGenerationZip(stat_var)
zipcode_result


Failed to fetch data for ZIP code Zipcode (DCID Zipcode DCID): URL can't contain control characters. '/stat/series?place=Zipcode DCID&stat_var=Count_Person' (found at least ' ')


,ZIPCode,DCID,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,00601,zip/00601,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16834.00
1,00602,zip/00602,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37642.00
2,00603,zip/00603,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,49075.00
3,00606,zip/00606,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5590.00
4,00610,zip/00610,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25542.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33966,96049,zip/96049,101.00,109.00,89.00,90.00,76.00,28.00,46.00,87.00,90.00,87.00,0.00,0.00
33967,98205,zip/98205,11512.00,11353.00,11845.00,11957.00,12644.00,12783.00,13518.00,14360.00,15394.00,15784.00,0.00,0.00
33968,98929,zip/98929,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
33969,99164,zip/99164,4.00,3.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# try pull dcid directly from datacommon


failed,idk what should i do